In [2]:
data = {'L0 Domain': 'L0 Domain',
 'L1 Category': 'L1 Category',
 'L2 Category': 'L2 Category',
 'L3 Sub-Category': 'L3 Sub-Category',
 'L4 Product Enum-Code': 'L4 Product Enum-Code',
 'Product Name': 'Product Name',
 'Product Quantity': 'Product Quantity',
 'UOM': 'UOM',
 'MRP': 'MRP',
 'Price': 'Price',
 'Product Description': 'Product Description',
 'SKU Code': 'SKU Code',
 'Manufacturer': 'Manufacturer',
 'Country Of Origin': 'Country Of Origin',
 'Customer Care Contact': 'Customer Care Contact',
 'Time to Ship': 'Time to Ship',
 'Returnable (Y/N)': 'Returnable (Y/N)',
 'Cancellable (Y/N)': 'Cancellable (Y/N)',
 'COD available (Y/N)': 'COD available (Y/N)',
 'Images': 'Images',
 'Net Quantity': 'Net Quantity',
 'Brand': 'Brand',
 'Pack Quantity': 'Pack Quantity',
 'Pack Size': 'Pack Size',
 'Images / Video': 'Images / Video',
 'UPC/EAN': 'UPC/EAN',
 'FSSAI no': 'FSSAI no',
 'Preservatives (Y/N)': 'Preservatives (Y/N)',
 'Preservatives (details)': 'Preservatives (details)',
 'Flavours & Spices': 'Flavours & Spices',
 'Ready to cook (Y/N)': 'Ready to cook (Y/N)',
 'Ready to eat (Y/N)': 'Ready to eat (Y/N)',
 'Rice grain type': 'Rice grain type',
 'Recommended Age': 'Recommended Age',
 'Scented/ Flavour': 'Scented/ Flavour',
 'Herbal/ Ayurvedic': 'Herbal/ Ayurvedic',
 'Theme/ Occasion Type': 'Theme/ Occasion Type',
 'Hair Type': 'Hair Type',
 'Mineral Source': 'Mineral Source',
 'Caffeine Content': 'Caffeine Content',
 'Capacity': 'Capacity',
 'Composition': 'Composition',
 'Benefits': 'Benefits',
 'Usage': 'Usage',
 'Other details': 'Other details',
 'Marketed By': 'Marketed By',
 'Images 3': 'Images 3',
 'Images 4': 'Images 4',
 'Images 5': 'Images 5',
 'Images 6': 'Images 6',
 'Images 7': 'Images 7',
 'Images 8': 'Images 8',
 'Images 9': 'Images 9',
 'Baby Weight': 'Baby Weight',
 'Absorption Duration (in Hrs)': 'Absorption Duration (in Hrs)',
 'Features': 'Features',
 'Images 10': 'Images 10',
 'Care Instruction': 'Care Instruction',
 'Ingredients': 'Ingredients',
 'Specification': 'Specification',
 'Package Contains': 'Package Contains',
 'About': 'About'}

from enum import Enum

Columns = Enum('Columns', data)

In [3]:
from pydantic import BaseModel
from typing import List, Optional, Dict

class MultiClassPrediction(BaseModel):
    """
    Class for a multi-class label prediction.
    """

    related_columns: List[Columns]

In [4]:
import os
from dotenv import load_dotenv

load_dotenv(
    "../ops/.env"
)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


import instructor
from openai import OpenAI

In [5]:
client = instructor.patch(OpenAI())


def multi_classify(data: str) -> MultiClassPrediction:
    """Perform multi-label classification on the input text."""
    return client.chat.completions.create(
        model="gpt-4",
        response_model=MultiClassPrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following requirement to strongly related column names: {data}",
            },
        ],
    )  # type: ignore

In [5]:
result = multi_classify("All products should have four image columns")
result.model_dump_json()

'{"related_columns":["Images","Images 3","Images 4","Images 5"]}'

In [7]:
import json

class SQLQUERYMODEL(BaseModel):
    """
    Class for an SQL QUERY 
    """
    queries: List[str]

def generate_sql_queries(requirement, column_names, table_name, client: OpenAI, model: str = "gpt-4"):
    """
    Extracts the columns from the requirement
    """
    response = client.chat.completions.create(
        model=model,
        response_model=SQLQUERYMODEL,
        messages=[
            {
                "role": "user",
                "content": f"Write SQL query to check the requirement {requirement} is true or false. The column names are {column_names} and table name is {table_name}. Write one query to check if all columns exist and another to return corresponding row numbers contain null.",
            },
        ],
    )
    return json.loads(response.model_dump_json())
    
generate_sql_queries("All products should have four image columns", "Images 1, Images 2, Images 3, Images 4", "products", client)

{'queries': ["SELECT column_name FROM information_schema.columns WHERE table_name = 'products' AND column_name IN ('Images 1', 'Images 2', 'Images 3', 'Images 4');",
  'SELECT ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS RowNum, * FROM products WHERE `Images 1` IS NULL OR `Images 2` IS NULL OR `Images 3` IS NULL OR `Images 4` IS NULL']}